# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [25]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [26]:
# Your code here; create the connection
conn= sqlite3.connect('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [27]:
# Your code here
p = """
SELECT
   customerNumber,
   contactLastName,
   contactFirstName
FROM customers
WHERE customerNumber IN (SELECT customerNumber
       FROM orders
       WHERE orderDate = '2003-01-31')
"""

pd.read_sql(p, conn)

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [28]:
p= """
SELECT  p.productName,
    (
        SELECT SUM(od.quantityOrdered)
        FROM orderdetails od
        WHERE od.productCode = p.productCode
    ) AS total_items_sold
FROM products p
ORDER BY total_items_sold DESC;
"""

pd.read_sql(p, conn)


,productName,total_items_sold
0,1992 Ferrari 360 Spider red,1808.0
1,1937 Lincoln Berline,1111.0
2,American Airlines: MD-11S,1085.0
3,1941 Chevrolet Special Deluxe Cabriolet,1076.0
4,1930 Buick Marquette Phaeton,1074.0
...,...,...
105,1911 Ford Town Car,832.0
106,1936 Mercedes Benz 500k Roadster,824.0
107,1970 Chevy Chevelle SS 454,803.0
108,1957 Ford Thunderbird,767.0


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [29]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
p= """  
SELECT 
    p.productName,
    COUNT(DISTINCT pc.customerNumber) AS NumCustomers
FROM products p
JOIN (
    SELECT 
        od.productCode,
        o.customerNumber
    FROM orderdetails od
    JOIN orders o ON od.orderNumber = o.orderNumber
) AS pc ON p.productCode = pc.productCode
GROUP BY p.productName
ORDER BY NumCustomers DESC;
"""

pd.read_sql(p, conn)


,productName,NumCustomers
0,1992 Ferrari 360 Spider red,40
1,Boeing X-32A JSF,27
2,1972 Alfa Romeo GTA,27
3,1952 Alpine Renault 1300,27
4,1934 Ford V8 Coupe,27
...,...,...
104,1958 Chevy Corvette Limited Edition,19
105,2002 Chevy Corvette,18
106,1969 Chevrolet Camaro Z28,18
107,1952 Citroen-15CV,18


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [30]:
# Your code here
p = """
SELECT DISTINCT 
    e.employeeNumber,
    e.firstName,
    e.lastName,
    oof.city,
    e.officeCode
FROM employees e
JOIN customers c ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN orders o ON c.customerNumber = o.customerNumber
JOIN orderdetails od ON o.orderNumber = od.orderNumber
JOIN products p ON od.productCode = p.productCode
JOIN offices oof ON e.officeCode = oof.officeCode
WHERE p.productCode IN (
    SELECT od.productCode
    FROM orderdetails od
    JOIN orders o ON od.orderNumber = o.orderNumber
    JOIN customers c ON o.customerNumber = c.customerNumber
    GROUP BY od.productCode
    HAVING COUNT(DISTINCT c.customerNumber) < 20
);
"""

pd.read_sql(p, conn)

,employeeNumber,firstName,lastName,city,officeCode
0,1501,Larry,Bott,London,7
1,1166,Leslie,Thompson,San Francisco,1
2,1612,Peter,Marsh,Sydney,6
3,1611,Andy,Fixter,Sydney,6
4,1401,Pamela,Castillo,Paris,4
5,1323,George,Vanauf,NYC,3
6,1165,Leslie,Jennings,San Francisco,1
7,1216,Steve,Patterson,Boston,2
8,1286,Foon Yue,Tseng,NYC,3
9,1370,Gerard,Hernandez,Paris,4


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [31]:
# Your code here
p = """ 
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(DISTINCT c.customerNumber) AS NumCustomers
FROM employees e 
JOIN (SELECT DISTINCT c.customerNumber, c.salesRepEmployeeNumber
                  FROM customers c
                  JOIN orders o ON c.customerNumber = o.customerNumber
                  WHERE c.creditLimit > 15000)
                  AS fc ON e.employeeNumber = fc.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY NumCustomers DESC;
"""

pd.read_sql(p, conn)

DatabaseError: Execution failed on sql ' 
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(DISTINCT c.customerNumber) AS NumCustomers
FROM employees e 
JOIN (SELECT DISTINCT c.customerNumber, c.salesRepEmployeeNumber
                  FROM customers c
                  JOIN orders o ON c.customerNumber = o.customerNumber
                  WHERE c.creditLimit > 15000)
                  AS fc ON e.employeeNumber = fc.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY NumCustomers DESC;
': no such column: c.customerNumber

In [32]:
p = """ 
SELECT 
    e.employeeNumber, 
    e.firstName, 
    e.lastName, 
    COUNT(DISTINCT fc.customerNumber) AS NumCustomers
FROM employees e 
JOIN (
    SELECT DISTINCT c.customerNumber, c.salesRepEmployeeNumber
    FROM customers c
    JOIN orders o ON c.customerNumber = o.customerNumber
    WHERE c.creditLimit > 15000
) AS fc ON e.employeeNumber = fc.salesRepEmployeeNumber
GROUP BY e.employeeNumber, e.firstName, e.lastName
ORDER BY NumCustomers DESC;
"""
pd.read_sql(p, conn)


,employeeNumber,firstName,lastName,NumCustomers
0,1401,Pamela,Castillo,10
1,1504,Barry,Jones,9
2,1323,George,Vanauf,8
3,1501,Larry,Bott,8
4,1370,Gerard,Hernandez,7
5,1165,Leslie,Jennings,6
6,1188,Julie,Firrelli,6
7,1216,Steve,Patterson,6
8,1286,Foon Yue,Tseng,6
9,1337,Loui,Bondur,6


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!